# PDF Summary

In [1]:
#!pip install --upgrade sagemaker --quiet
!pip install langchain==0.0.148 --quiet
#!pip install faiss-cpu --quiet

In [2]:
import time
import sagemaker, boto3, json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()
model_version = "*"

In [3]:
import boto3, json
from sagemaker.session import Session

In [4]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

In [5]:
endpoint_name = 'jumpstart-dft-hf-llm-falcon-7b-instruct-bf16'

In [6]:
from langchain import SagemakerEndpoint

In [7]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler

In [8]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode('utf-8')
      
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

content_handler = ContentHandler()

In [9]:
parameters = {
    "max_length": 200,
    "num_return_sequences": 1,
    "top_k": 250,
    "top_p": 0.95,
    "do_sample": False,
    "temperature": 1,
}

In [10]:
llm = SagemakerEndpoint(
    endpoint_name = endpoint_name, 
    region_name = aws_region, 
    model_kwargs = parameters,
    content_handler = content_handler
)

## PDF Summary

In [11]:
!pip install PyPDF2 --quiet

In [12]:
import PyPDF2
from io import BytesIO

In [13]:
sess = sagemaker.Session()
s3_bucket = sess.default_bucket()
s3_prefix = 'docs'

In [14]:
#s3_file_name = 'sample-blog.pdf'
s3_file_name = '2016-3series.pdf'
#s3_file_name = 'gen-ai-aws.pdf'

In [15]:
s3r = boto3.resource("s3")
doc = s3r.Object(s3_bucket, s3_prefix+'/'+s3_file_name)
#doc = s3r.Object(, '/docs/sample.pdf')
        
contents = doc.get()['Body'].read()
reader = PyPDF2.PdfReader(BytesIO(contents))
        
raw_text = []
for page in reader.pages:
    raw_text.append(page.extract_text())
contents = '\n'.join(raw_text)  

In [16]:
new_contents = str(contents[:8000]).replace("\n"," ") 
print('new_contents: ', new_contents)

new_contents:  Owner's Manual for Vehicle The Ultimate Driving Machine® THE BMW 3 SERIES SEDAN. OWNER'S MANUAL. Contents A-Z Online Edition for Part no. 01 40 2 960 440 - II/15  3 Series Owner's Manual for Vehicle Thank you for choosing a BMW. The more familiar you are with your vehicle, the better control you will have on the road. We therefore strongly suggest: Read this Owner's Manual before starting off in your new BMW. Also use the Integrated Owner's Manual in your vehicle. It con‐ tains important information on vehicle operation that will help you make full use of the technical features available in your BMW. The manual also contains information designed to en‐ hance operating reliability and road safety, and to contribute to maintaining the value of your BMW. Any updates made after the editorial deadline for the printed or Integrated Owner's Manual are found in the appendix of the printed Quick Reference for the vehicle. Supplementary information can be found in the additional b

In [30]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)
texts = text_splitter.split_text(new_contents) 

In [26]:
texts[0]

"Owner's Manual for Vehicle The Ultimate Driving Machine® THE BMW 3 SERIES SEDAN. OWNER'S MANUAL. Contents A-Z Online Edition for Part no. 01 40 2 960 440 - II/15  3 Series Owner's Manual for Vehicle Thank you for choosing a BMW. The more familiar you are with your vehicle, the better control you will have on the road. We therefore strongly suggest: Read this Owner's Manual before starting off in your new BMW. Also use the Integrated Owner's Manual in your vehicle. It con‐ tains important information on vehicle operation that will help you make full use of the technical features available in your BMW. The manual also contains information designed to en‐ hance operating reliability and road safety, and to contribute to maintaining the value of your BMW. Any updates made after the editorial deadline for the printed or Integrated Owner's Manual are found in the appendix of the printed Quick Reference for the vehicle. Supplementary information can be found in the additional bro‐ chures in 

In [32]:
texts[1]

"can be found in the additional bro‐ chures in the onboard literature. We wish you a safe and enjoyable ride. BMW AG   The Owner's Manual is available in many countries as an app. Additional information on the Internet: www.bmw.com/bmw_drivers_guide Online Edition for Part no. 01 40 2 960 440 - II/15 © 2015 Bayerische Motoren Werke Aktiengesellschaft Munich, Germany Reprinting, including excerpts, only with the written consent of BMW AG, Munich. US English II/15, 03 15 490 Printed on environmentally friendly paper, bleached without chlorine, suitable for recycling. Online Edition for Part no. 01 40 2 960 440 - II/15 Addendum ADDENDUM TO OWNER'S MANUAL We wanted to provide you with some updates  and clarifications with respect to the printed  BMW Owner's Manual. These updates and  clarifications will supersede the materials con- tained in that document.  1.Where the terms “service center,” “the ser- vice center,” “your service center,” “service  specialist,” or “service” are used in the

In [33]:
from langchain.docstore.document import Document
docs = [
    Document(
        page_content=t
    ) for t in texts[:3]
]
print(docs)

[Document(page_content="Owner's Manual for Vehicle The Ultimate Driving Machine® THE BMW 3 SERIES SEDAN. OWNER'S MANUAL. Contents A-Z Online Edition for Part no. 01 40 2 960 440 - II/15  3 Series Owner's Manual for Vehicle Thank you for choosing a BMW. The more familiar you are with your vehicle, the better control you will have on the road. We therefore strongly suggest: Read this Owner's Manual before starting off in your new BMW. Also use the Integrated Owner's Manual in your vehicle. It con‐ tains important information on vehicle operation that will help you make full use of the technical features available in your BMW. The manual also contains information designed to en‐ hance operating reliability and road safety, and to contribute to maintaining the value of your BMW. Any updates made after the editorial deadline for the printed or Integrated Owner's Manual are found in the appendix of the printed Quick Reference for the vehicle. Supplementary information can be found in the add

In [34]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="stuff")
chain.run(docs)

"\nThe Owner's Manual for the 3 Series Sedan provides information on the vehicle's"

In [35]:
from langchain.prompts import PromptTemplate

prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY """


In [36]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
chain.run(docs)

"\nThe Owner's Manual for the 3 Series Sedan is a comprehensive guide to the vehicle"